<a href="https://colab.research.google.com/github/inokchoi/ComputerVision/blob/main/CNN_99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
tf.set_random_seed(777)
 
mnist = input_data.read_data_sets("MINST_data/",one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

#모델 저장 경로
save_file = './model'
#TensorBoard log 저장 경로
logs_path = './logs'

In [ ]:
keep_prob = tf.placeholder(tf.float32)

#28*28이미지 -> X를 784(28*28)
X = tf.placeholder(tf.float32, [None, 784])
#이미지를 받아올 X_img
X_img = tf.reshape(X, [-1, 28, 28, 1])
#Label을 받아올 Y
Y = tf.placeholder(tf.float32, [None,10])

W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
 
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
 
W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1],padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 128*4*4])
 
W4 = tf.get_variable("W4", shape=[128*4*4, 625], initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
 
W5 = tf.get_variable("W5", shape = [625,10], initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4,W5) + b5

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
    
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
            c, _, summary = sess.run([cost, optimizer, merged_summary_op], feed_dict=feed_dict)
            summary_writer.add_summary(summary, epoch * total_batch + i)
            avg_cost += c/total_batch
 
        print('Epoch;', '%04d' % (epoch +1), 'cost =', '{:.9f}'.format(avg_cost)) 
    print('Learning Finished')
    print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))
    
    # Save the model
    saver.save(sess, save_file)

Epoch; 0001 cost = 0.365652760
Epoch; 0002 cost = 0.102024115
Epoch; 0003 cost = 0.073605365
Epoch; 0004 cost = 0.060561172
Epoch; 0005 cost = 0.050102010
Epoch; 0006 cost = 0.045606690
Epoch; 0007 cost = 0.043939578
Epoch; 0008 cost = 0.038386950
Epoch; 0009 cost = 0.035203932
Epoch; 0010 cost = 0.035286386
Epoch; 0011 cost = 0.029418148
Epoch; 0012 cost = 0.030336113
Epoch; 0013 cost = 0.028220017
Epoch; 0014 cost = 0.025865923
Epoch; 0015 cost = 0.025325084
Learning Finished
Accuracy: 0.9938


TensorBoard 실행을 위한 코드

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(logs_path):
  os.makedirs(logs_path)

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(logs_path))

get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-10-07 15:07:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.214.169.236, 52.54.237.49, 3.219.201.17, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.214.169.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  12.98M  46.7MB/s    in 0.3s    

2019-10-07 15:07:05 (46.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
https://471fc14d.ngrok.io
